# T1526 - Cloud Service Discovery
An adversary may attempt to enumerate the cloud services running on a system after gaining access. These methods can differ from platform-as-a-service (PaaS), to infrastructure-as-a-service (IaaS), or software-as-a-service (SaaS). Many services exist throughout the various cloud providers and can include Continuous Integration and Continuous Delivery (CI/CD), Lambda Functions, Azure AD, etc. They may also include security services, such as AWS GuardDuty and Microsoft Defender for Cloud, and logging services, such as AWS CloudTrail and Google Cloud Audit Logs.

Adversaries may attempt to discover information about the services enabled throughout the environment. Azure tools and APIs, such as the Azure AD Graph API and Azure Resource Manager API, can enumerate resources and services, including applications, management groups, resources and policy definitions, and their relationships that are accessible by an identity.(Citation: Azure - Resource Manager API)(Citation: Azure AD Graph API)

For example, Stormspotter is an open source tool for enumerating and constructing a graph for Azure resources and services, and Pacu is an open source AWS exploitation framework that supports several methods for discovering cloud services.(Citation: Azure - Stormspotter)(Citation: GitHub Pacu)

Adversaries may use the information gained to shape follow-on behaviors, such as targeting data or credentials from enumerated services or evading identified defenses through [Disable or Modify Tools](https://attack.mitre.org/techniques/T1562/001) or [Disable or Modify Cloud Logs](https://attack.mitre.org/techniques/T1562/008).

## Atomic Tests

### Atomic Test #1 - Azure - Dump Subscription Data with MicroBurst
Upon successful execution, this test will enumerate all resources that are contained within a valid Azure subscription. 
The resources enumerated will display on screen, as well as several csv files and folders will be output to a specified directory, listing what resources were discovered by the script. 
See https://dev.to/cheahengsoon/enumerating-subscription-information-with-microburst-35a1

**Supported Platforms:** iaas:azure
#### Dependencies:  Run with `powershell`!
##### Description: The Get-AzDomainInfo script must exist in PathToAtomicsFolder\..\ExternalPayloads.

##### Check Prereq Commands:
```powershell
if (test-path "PathToAtomicsFolder\..\ExternalPayloads\Get-AzDomainInfo.ps1"){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
invoke-webrequest "https://raw.githubusercontent.com/NetSPI/MicroBurst/c771c665a2c71f9c5ba474869cd1c211ebee68fd/Az/Get-AzDomainInfo.ps1" -outfile "PathToAtomicsFolder\..\ExternalPayloads\Get-AzDomainInfo.ps1"

```
##### Description: The Az module must be installed.

##### Check Prereq Commands:
```powershell
try {if (Get-InstalledModule -Name Az -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name Az -Force

```

In [ ]:
Invoke-AtomicTest T1526 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
import-module "PathToAtomicsFolder\..\ExternalPayloads\Get-AzDomainInfo.ps1"
$Password = ConvertTo-SecureString -String "T1082Az" -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "None", $Password
Connect-AzAccount -Credential $Credential | out-null
Get-AzDomainInfo -folder $env:temp\T1526Test1 -subscription "None" -verbose
```

In [ ]:
Invoke-AtomicTest T1526 -TestNumbers 1

#### Cleanup: 
```powershell
remove-item $env:temp\T1526Test1 -recurse -force -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1526 -TestNumbers 1 -Cleanup

## Detection
Cloud service discovery techniques will likely occur throughout an operation where an adversary is targeting cloud-based systems and services. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities based on the information obtained.

Normal, benign system and network events that look like cloud service discovery may be uncommon, depending on the environment and how they are used. Monitor cloud service usage for anomalous behavior that may indicate adversarial presence within the environment.


## Shield Active Defense
### Decoy Network 
 Create a target network with a set of target systems, for the purpose of active defense.
 

 Decoy networks are comprised of multiple computing resources that can be used for defensive or deceptive purposes.  A decoy network can be used to safely perform dynamic analysis of suspected malicious code. A defender can also use a specially crafted decoy network to perform adversary engagement. 
#### Opportunity
There is an opportunity to introduce services in a decoy network to determine if an adversary notices and tries to learn more about them.
#### Use Case
A defender can use a decoy network and seed it with cloud services to see how an adversary might exploit those resources.
#### Procedures
Create an isolated network populated with decoy systems that can be used to study an adversary's tactics, techniques, and procedures (TTPs).
Use a segregated network to visit a compromised site.  If the machine becomes infected, allow the machine to remain on with internet access to see if an adversary engages and takes action on the system.
